Simple solution using client history
===============================

In this notebook I present a simple solution that predicts that the user will buy everything he has bought before.

Imports
-------

In [ ]:
import numpy as np
import pandas as pd

Reading the Input
-----------------
To avoid running out of memory, the other datasets weren't even used.

In [ ]:
orders_prior = pd.read_csv('../input/order_products__prior.csv')
orders_train = pd.read_csv('../input/order_products__train.csv')
orders = pd.read_csv('../input/orders.csv')

Stacking Prior and Train tables
--------------------------------

In [ ]:
order_products = pd.concat([orders_prior, orders_train], axis=0)
del orders_train, orders_prior

Concat products function (Change this!)
------------------------------
This function concatenates every product in the array.

In [ ]:
def products_concat(vet):
    out = ''
    
    #vet is a pd.Series
    for prod in vet:
        if prod > 0:
            out += str(int(prod)) + ' '
    
    if out != '':
        return out.rstrip()
    else:
        return 'None'

Combining all data
----------------------------------------------
Combine Orders with our Order_products table, and merge this by user_id using our concat function.

In [ ]:
all_data = order_products.merge(orders, on='order_id', how='outer')
del order_products

user_products = all_data.groupby('user_id').product_id.apply(products_concat)

Generating test set, which contains the user_id and every product he bought.

In [ ]:
test_set = all_data.loc[all_data.eval_set == 'test'][['order_id', 'user_id']]
test_set = test_set.join(user_products, on='user_id')

Only need these columns for the submission.

In [ ]:
submission = pd.DataFrame({'order_id': test_set.order_id, 'products': test_set.product_id})
submission.head()

Saving to file.

In [ ]:
submission.to_csv('simple_sub.csv', index=False)

Now it's your turn
------------------
Use feature engineering to produce more relevant features, use your creativity!

Some ideas:

 - Counting the number of times the user bought a product
 - Thresholds involving number of times bought / how long ago it was bought
 - Use the timestamp of the order to filter certain products